In [2]:
import pandas as pd

df = pd.read_csv('VOO Historical Data.csv')

df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,29/10/2024,534.77,533.12,535.82,531.75,3.03M,0.16%
1,28/10/2024,533.92,535.53,535.57,533.67,3.41M,0.31%
2,25/10/2024,532.26,534.65,537.26,531.41,4.33M,-0.04%
3,24/10/2024,532.47,533.17,533.18,530.04,4.48M,0.23%
4,23/10/2024,531.27,534.28,534.72,528.02,5.23M,-0.91%


In [6]:
# Find the best day of the month to buy VOO

# for every month, find the day with the lowest price

df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Year'] = df['Date'].dt.year

df.head()

,Date,Price,Open,High,Low,Vol.,Change %,Month,Day,Year
0,2024-10-29,534.77,533.12,535.82,531.75,3.03M,0.16%,10,29,2024
1,2024-10-28,533.92,535.53,535.57,533.67,3.41M,0.31%,10,28,2024
2,2024-10-25,532.26,534.65,537.26,531.41,4.33M,-0.04%,10,25,2024
3,2024-10-24,532.47,533.17,533.18,530.04,4.48M,0.23%,10,24,2024
4,2024-10-23,531.27,534.28,534.72,528.02,5.23M,-0.91%,10,23,2024


In [13]:
# group by month and then find the day with the lowest price

df.groupby(['Year','Month'])['Price'].min().reset_index()

# find the day the price is the lowest for each month

# df.groupby(['Year', 'Month'])['Price']


,Year,Month,Price
0,2010,9,101.78
1,2010,10,103.84
2,2010,11,107.96
3,2010,12,110.62
4,2011,1,116.12
...,...,...,...
165,2024,6,485.15
166,2024,7,494.78
167,2024,8,475.20
168,2024,9,496.64


In [18]:
import pandas as pd

def analyze_best_buying_day(df):
    """
    Analyze the best day to buy based on lowest prices per month.
    Returns the average best day to buy across all months.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with date and price information
    
    Returns:
    tuple: (average best day, DataFrame with lowest price days per month)
    """
    # Convert date string to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Extract month and day
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Year'] = df['Date'].dt.year
    
    # Group by month and find the day with lowest price
    monthly_lowest = df.loc[df.groupby(['Year', 'Month'])['Price'].idxmin()]
    
    # Calculate the average day
    average_best_day = monthly_lowest['Day'].mean()
    
    # Sort by date for better readability
    monthly_lowest = monthly_lowest.sort_values('Date')
    
    # Select relevant columns for display
    result_df = monthly_lowest[['Date', 'Day', 'Price', 'Month']]
    
    return average_best_day, result_df

# Example usage:
# Assuming your data is in a DataFrame called 'df'
# df = pd.read_csv('your_data.csv')
avg_day, monthly_data = analyze_best_buying_day(df)

print(f"Average best day to buy: {avg_day:.2f}")
print(monthly_data)



Average best day to buy: 12.40
           Date  Day   Price  Month
3586 2010-09-10   10  101.78      9
3570 2010-10-04    4  103.84     10
3539 2010-11-16   16  107.96     11
3529 2010-12-01    1  110.62     12
3506 2011-01-04    4  116.12      1
...         ...  ...     ...    ...
103  2024-06-03    3  485.15      6
67   2024-07-25   25  494.78      7
60   2024-08-05    5  475.20      8
37   2024-09-06    6  496.64      9
18   2024-10-03    3  521.84     10

[170 rows x 4 columns]


In [ ]:
import pandas as pd

def analyze_buying_strategies(df):
    """
    Compare different buying strategies: buying on 1st day, 13th day, and lowest price day.
    
    Parameters:
    df (pandas.DataFrame): DataFrame with date and price information
    
    Returns:
    tuple: (average best day, strategy comparison DataFrame)
    """
    # Convert date string to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Extract month and day
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Year'] = df['Date'].dt.year
    
    # Initialize results storage
    results = []
    
    # Analyze each month
    for (year, month), month_data in df.groupby(['Year', 'Month']):
        # Find lowest price day
        lowest_price_row = month_data.loc[month_data['Price'].idxmin()]
        
        # Try to get 1st day price (if available)
        first_day_price = month_data[month_data['Day'] == 1]['Price'].iloc[0] if len(month_data[month_data['Day'] == 1]) > 0 else None
        
        # Try to get 13th day price (if available)
        day_13_price = month_data[month_data['Day'] == 13]['Price'].iloc[0] if len(month_data[month_data['Day'] == 13]) > 0 else None
        
        results.append({
            'Year': year,
            'Month': month,
            'Lowest_Day': lowest_price_row['Day'],
            'Lowest_Price': lowest_price_row['Price'],
            'First_Day_Price': first_day_price,
            'Day_13_Price': day_13_price,
            'First_Day_Difference': (first_day_price - lowest_price_row['Price']) if first_day_price is not None else None,
            'Day_13_Difference': (day_13_price - lowest_price_row['Price']) if day_13_price is not None else None
        })
    
    # Convert results to DataFrame
    analysis_df = pd.DataFrame(results)
    
    # Calculate average day of lowest price
    average_best_day = analysis_df['Lowest_Day'].mean()
    
    # Calculate strategy comparisons
    total_first_day_diff = analysis_df['First_Day_Difference'].sum()
    total_day_13_diff = analysis_df['Day_13_Difference'].sum()
    
    # Add summary statistics
    print(f"Average best day to buy: Day {average_best_day:.1f} of the month")
    print(f"\nTotal price difference if buying on 1st vs lowest price day: ${total_first_day_diff:.2f}")
    print(f"Total price difference if buying on 13th vs lowest price day: ${total_day_13_diff:.2f}")
    
    # Calculate percentage differences
    avg_first_day_pct = (analysis_df['First_Day_Difference'] / analysis_df['Lowest_Price'] * 100).mean()
    avg_day_13_pct = (analysis_df['Day_13_Difference'] / analysis_df['Lowest_Price'] * 100).mean()
    
    print(f"\nAverage percentage difference for 1st day strategy: {avg_first_day_pct:.2f}%")
    print(f"Average percentage difference for 13th day strategy: {avg_day_13_pct:.2f}%")
    
    return average_best_day, analysis_df.sort_values(['Year', 'Month'])

# Example usage:
# df = pd.read_csv('your_data.csv')
avg_day, analysis = analyze_buying_strategies(df)
print("\nDetailed monthly analysis:")
print(analysis)